In [82]:
import pandas
import json
import h2o
import os
import random
from pathlib import Path
from h2o.automl import H2OAutoML

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/ramses/Documentos/proyects/web-mining/NewsNabber/.venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0a0x1w_j
  JVM stdout: /tmp/tmp0a0x1w_j/h2o_ramses_started_from_python.out
  JVM stderr: /tmp/tmp0a0x1w_j/h2o_ramses_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_ramses_b8tmv9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.914 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [5]:
def load_articles() -> list:
    articles = []
    for file_path in os.listdir("./data"):
        try:
            data_page = open(Path("data", file_path))
            articles_by_page = json.load(data_page)
            articles = articles + articles_by_page
        except:
            print(f"No se pudo cargar el arcivo: {file_path}")
    
    return articles

In [6]:
def preprocess_text(text: str):
    return []

# LOAD DATASET

In [76]:
dataframe = pandas.DataFrame(load_articles())

# ADD NEW COLUMNS TO DATAFRAME 

In [77]:
dataframe['date'] = pandas.to_datetime(dataframe['date'], errors="coerce")
dataframe['only_date'] = dataframe['date'].dt.date
dataframe['only_weekday'] = dataframe['date'].dt.isocalendar().week

In [78]:
dataframe.groupby(['only_weekday'])['only_weekday'].count()

only_weekday
31    21
32    22
33    69
34    64
Name: only_weekday, dtype: int64

Se decide tomar N registros de manera random para cada semana de articulos

In [124]:
def get_random_articles_by_weekday(df: pandas.DataFrame):
    weekdays = df['only_weekday'].unique().tolist()
    total_articles = pandas.DataFrame()
    
    for weekday in weekdays:
        df_articles = df.loc[df['only_weekday'] == weekday]
        indexes = random.sample(df_articles.index.values.tolist(), 5)
        df_articles = df_articles.filter(items=indexes, axis=0)
        
        total_articles = pandas.concat([total_articles, df_articles])
    
    return total_articles

In [126]:
dataframe_article = get_random_articles_by_weekday(dataframe)
dataframe_article

,date,description,title,category,only_date,only_weekday
116,2023-08-22 14:54:13-03:00,La Comisión de Legislación General se reúne pa...,,el-pais,2023-08-22,34
10,2023-08-25 00:01:00-03:00,El Banco Central tomó la decisión de que los p...,,economia,2023-08-25,34
77,2023-08-21 00:01:00-03:00,"Como todos los años desde 1996, y en la ciudad...",,el-pais,2023-08-21,34
3,2023-08-25 00:01:00-03:00,Las ventas en supermercados medidas a precios ...,,economia,2023-08-25,34
0,2023-08-25 19:46:52-03:00,A la espera de los\nanuncios del ministro de E...,,economia,2023-08-25,34
25,2023-08-16 00:01:00-03:00,La postal del domingo fue para el establishmen...,,economia,2023-08-16,33
73,2023-08-18 13:52:23-03:00,La candidata a diputada nacional por La Libert...,,el-pais,2023-08-18,33
56,2023-08-18 18:13:21-03:00,"La secretaria de Energía, Flavia Royón, antici...",,economia,2023-08-18,33
66,2023-08-19 04:18:38-03:00,Las señales que emite el voto a Javier Milei s...,,el-pais,2023-08-19,33
125,2023-08-17 00:01:00-03:00,Sergio Massa había hablado por última vez la n...,,el-pais,2023-08-17,33


In [ ]:
def generate_model(df_train: pandas.DataFrame, df_test: pandas.DataFrame):
    train = h2o.H2OFrame(df_train)
    test = h2o.H2OFrame(df_test)

    x = train.columns
    y = "category"
    x.remove(y)

    train[y] = train[y].asfactor()
    test[y] = test[y].asfactor()

    auto_machine_learning = H2OAutoML(max_models=20, seed=1, max_runtime_secs=60*2)
    auto_machine_learning.train(x=x, y=y, training_frame=train)